# Chap07 - 텐서플로 추상화와 간소화

> 추상화가 무엇이며 왜 유용한지 알아보고 텐서플로(TensorFlow)의 몇몇 대중적인 추상화 라이브러리를 간단하게 살펴보도록 하자.

## 7.1 개요

**추상화(abstraction)**이란 코드를 특정한 목적으로 일반화하여 기존 코드 **'위에 올라가는'** 코드의 계층을 의미한다. 관련있는 High-Level 기능을 묶는 방식의 재구성을 통해 코드를 감싸서 추상화한다. 따라서 쉽게 코딩할 수 있고, 가독성이 좋으며, 코드가 간소화된다.

### 7.1.1 추상화 라이브러리 둘러보기

텐서플로(TensorFlow)에서 사용할 수 있는 인기있는 추상화 라이브러리는 다음과 같다고 한다.

- `tf.estimator`
- TFLearn
- TF-Slim
- Keras

TFLearn은 별도의 설치가 필요하며 `tf.estimator`과 TF-Slim([`tf.contrib.slim`](https://github.com/tensorflow/tensorflow/tree/r1.8/tensorflow/contrib/slim))은 텐서플로에 포함되어 있다. Keras(케라스)는 2017년 구글의 공식 후원을 받아 1.1버전 부터 `tf.contrib.keras`안에 들어왔으며, 현재 이글을 작성하는 시점인 2018.06.27의 1.8버전에는 `tf.keras`로 변경됭 텐서플로의 한 부분으로 자리를 잡았다. `contrib`안의 라이브러리들은 **기부된(contributed)** 것을 의미하며 정식으로 반영되기 전까지 테스트가 더 필요하다는 것을 의미한다. 

`tf.estimator`은 Python의 대표적인 머신러닝 모듈인 [Scikit-Learn](http://scikit-learn.org/stable/index.html)(`sklearn`)의 스타일처럼 복잡한 딥러닝 모델을 쉽게 작성할 수 있도록 해주는 라이브러리다. Keras와 마찬가지로 [`tf.contrib.learn`](https://github.com/tensorflow/tensorflow/tree/r1.8/tensorflow/contrib/learn)으로 텐서플로에 들어왔으며, 현재 1.8버전(2018.06.27)에는 `tf.estimator`로 옮겨졌으며 `tf.contrib.learn`은 유지되고 있지만 사용할 경우 `deprecated` 경고가 나타난다.

TF-Slim은 주로 복잡한 합성곱 신경망(Convolutional Networks)을 쉽게 설계할 수 있도록 만들어졌으며 다양한 Pre-Trained 모델을 제공해 학습 시간을 단축시킬 수 있다. 

## 7.2 tf.estimator

`tf.estimator`는 `sklearn`과 유사하게 `Estimator`(모델들을 부르는 용어)로 동작하는데 이것을 이용하면, 빠르게 모델을 학습할 수 있다. 대표적인 Estimator들은 아래와 같으며, [`tf.estimator`](https://www.tensorflow.org/api_docs/python/tf/estimator)에서 다양한 Estimator들을 살펴볼 수 있다.

| Estimator                                                    | 설명                                                         |
| ------------------------------------------------------------ | ------------------------------------------------------------ |
| [`LinearRegressor()`](https://www.tensorflow.org/api_docs/python/tf/estimator/LinearRegressor) | 주어진 관측치에 대해 Linear regression을 이용해 예측         |
| [`LinearClassifier()`](https://www.tensorflow.org/api_docs/python/tf/estimator/LinearClassifier) | Linear Model을 이용한 다중분류(multi-classification), 클래스가 2개일 경우 binary classification |
| [`DNNRegressor()`](https://www.tensorflow.org/api_docs/python/tf/estimator/DNNRegressor) | TensorFlow DNN 모델을 이용한 regression                      |
| [`DNNClassifier()`](https://www.tensorflow.org/api_docs/python/tf/estimator/DNNClassifier) | TensorFlow DNN 모델을 이용한 classification                  |

`tf.contrib.learn`에서 `tf.estimator`로 옮겨올때 아래의 `Estimator`는 반영이 되지 않았다.

- `DynamicRnnEstimator`: Consider a custom `model_fn`.
- `KMeansClustering`: Use `tf.contrib.factorization.KMeansClustering`.
- `LogisticRegressor`: Not supported. Instead, use `binary_classification_head` with a custom `model_fn`, or with `DNNEstimator`.
- `StateSavingRnnEstimator`: Consider a custom `model_fn`.
- SVM: Consider a custom `model_fn`.
- `LinearComposableModel` and `DNNComposableModel`: Not supported. Consider `tf.contrib.estimator.DNNEstimator`, or write a custom model_fn.
- `MetricSpec`: Deprecated. For adding custom metrics to canned Estimators, use `tf.contrib.estimator.add_metrics`.

위의 Estimator를 사용하는 방법은 다음과 같다.

1. 클래스 **인스턴스화**(instance)
    - `model = tf.estimator.<some_Estimator>()`
2. 학습 데이터를 사용해 모델을 **학습** 
    - `model.train()`
3. 학습된 모델을 **평가**(evaluate)
    - `model.evaluate()`
4. 테스트 데이터를 이용해 결과를 **예측**(predict)
    - `model.predict()`

### 7.2.1 선형회귀 - Linear Regression

선형회귀(linear regression)은 데이터 집합 $\left\{ y_i, x_{i1}, \dots , x_{ip} \right\}_{i=1}^{n}$ 에 대해, 종속변수 $y_i$와 $p$개의 설명변수 $x_i$ 사이의 선형 관계를 모델링한다(출처: [wikipedia](https://ko.wikipedia.org/wiki/%EC%84%A0%ED%98%95_%ED%9A%8C%EA%B7%80)).

$$
y_i = w_1 x_{i1} + \cdots + w_{p} x_{ip} + \epsilon_i = \mathbf{x}_{i}^{T} \mathbf{W} + \epsilon_i, \quad i = 1, \dots , n
$$

위의 선형회귀 모델을 Scikit-Learn(사이킷런)에서 제공하는 [보스턴 하우징(Boston Housing)](http://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_boston.html)데이터에 적용해보도록 하자. 먼저 추상화 라이브러리를 사용하지 않은 텐서플로(TensorFlow)코드를 구현한 뒤, 이것을 `tf.estimator.LiearRegressor()`를 이용해 구현해 볼 것이다.

보스턴 하우징 데이터 설명은 다음과 같다. 종속변수(타겟변수)는 소유자가 거주 중인 주택 가격의 중간값(median)이다.

|      | Feature | 설명                                                      |
| ---- | ------- | --------------------------------------------------------- |
| 1    | CRIM    | 마을의 1인당 범죄율                                       |
| 2    | ZN      | 2만 5천 평방 피트 이상으로 구획된 택지의 비율             |
| 3    | INDUS   | 마을당 비상업 업무 지구의 비율                            |
| 4    | CHAS    | 찰스강 더미 변수 (인접: 1, 아니면: 0)                     |
| 5    | NOX     | 질소산화물 농도                                           |
| 6    | RM      | 주택당 평균 방의 수                                       |
| 7    | AGE     | 1940년 이전에 지어진 건물의 비율                          |
| 8    | DIS     | 5개의 보스턴 고용 센터까지의 가중 거리                    |
| 9    | RAD     | 방사형 고속도로까지의 접근성 지수                         |
| 10   | TAX     | 1만 달러당 부가가치세                                     |
| 11   | PTRATIO | 마을별 학생 대 교사 비율                                  |
| 12   | B       | $1000(\text{Bk}-0.63)^{2}$, $\text{Bk}$: 마을의 흑인 비율 |
| 13   | LSTAT   | 하위 계층의 비율(%)                                       |

먼저 Scikit-Learn에서 보스턴 하우징 데이터를 불러온다. 설명변수로 쓰일 데이터들을 [`StandardScaler()`](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html)를 이용해 정규화 해준다. `StandardScaler`의 수식은 다음과 같으며 *Z-Score*라고도 한다.

$$
z = \frac{\text{Data point} - \text{mean}}{\text{standard deviation}} = \frac{x-\mu}{\sigma}
$$

In [1]:
from sklearn import datasets, metrics, preprocessing

boston = datasets.load_boston()
x_data = preprocessing.StandardScaler().fit_transform(boston.data)
y_data = boston.target

print('x_data.shape :', x_data.shape)
print('y_data.shape :', y_data.shape)

x_data.shape : (506, 13)
y_data.shape : (506,)


#### Natvie TensorFlow

먼저 추상화 라이브러리를 사용하지 않고, 순수 텐서플로를 이용해 선형회귀를 모델링 해보도록 하자. 

In [2]:
import tensorflow as tf

In [3]:
x = tf.placeholder(tf.float64, shape=(None, 13))
y_true = tf.placeholder(tf.float64, shape=(None))

with tf.name_scope('inference'):
    w = tf.Variable(tf.zeros([1, 13], dtype=tf.float64, name='weights'))
    b = tf.Variable(0, dtype=tf.float64, name='bias')
    y_pred = tf.matmul(w, tf.transpose(x)) + b
    
with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.square(y_true-y_pred))  # MSE
    
with tf.name_scope('train'):
    learning_rate = 0.1
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    train = optimizer.minimize(loss)
    
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for step in range(200):
        MSE, _ = sess.run([loss, train], feed_dict={x: x_data, 
                                                    y_true: y_data})
    
        if (step+1) % 40 == 0:
            print('Step: {:2d}\t MSE: {:.5f}'.format(step+1, MSE))

Step: 40	 MSE: 22.30192
Step: 80	 MSE: 22.00200
Step: 120	 MSE: 21.93284
Step: 160	 MSE: 21.91024
Step: 200	 MSE: 21.90225


#### tf.estimator

이번에는 위와 똑같은 기능을 하는 코드를 `tf.estimator.LinearRegressor()`를 이용해 구현해 보도록 하자. 아래의 코드에서 확인할 수 있듯이 간단한 코드로 Linear Regression을 구현할 수 있다.

In [4]:
NUM_STEPS = 200
MINIBATCH_SIZE = 506

feature_column = [tf.feature_column.numeric_column(key='x', shape=13)]
train_input_fn = tf.estimator.inputs.numpy_input_fn(
        {'x': x_data}, y_data, batch_size=MINIBATCH_SIZE, shuffle=False)
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
        {'x': x_data}, y_data, batch_size=MINIBATCH_SIZE, shuffle=False)

reg = tf.estimator.LinearRegressor(
        feature_columns=feature_column,
        optimizer=tf.train.GradientDescentOptimizer(learning_rate=0.001), 
        model_dir='./model')

reg.train(input_fn=train_input_fn, steps=NUM_STEPS)
MSE = reg.evaluate(input_fn=eval_input_fn, steps=1)

print(MSE)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f0c3d9cd908>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into ./model/model.ckpt.
INFO:tensorflow:loss = 299626.34, step = 0
INF

### 7.2.2 DNN 분류기

이번에는 [2.4 - Softmax Regression](http://excelsior-cjh.tistory.com/149?category=940399)에서 구현한 MNIST 분류기를 `tf.estimator.DNNClassifier()`를 이용해 구현해보도록 하자.

In [5]:
import sys
import numpy as np
import tensorflow as tf


# MNIST 데이터 불러오기 위한 함수 정의
def mnist_load():
    (train_x, train_y), (test_x, test_y) = tf.keras.datasets.mnist.load_data()

    # Train set
    train_x = train_x.astype('float32') / 255.
    train_y = train_y.astype('int32')
    # Test set
    test_x = test_x.astype('float32') / 255.
    test_y = test_y.astype('int32')
    return (train_x, train_y), (test_x, test_y)

# MNIST 데이터 불러오기
(train_x, train_y), (test_x, test_y) = mnist_load()

# Define train_input_fn
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'x': train_x}, y=train_y, shuffle=True, batch_size=MINIBATCH_SIZE
)

In [7]:
NUM_STEPS = 5000
MINIBATCH_SIZ = 128

feature_columns = [tf.feature_column.numeric_column('x', shape=[28, 28])]

dnn = tf.estimator.DNNClassifier(
    feature_columns=feature_columns,
    hidden_units=[200],
    n_classes=10,
    optimizer=tf.train.ProximalAdagradOptimizer(learning_rate=0.2),
    model_dir='./model'
)

dnn.train(
    input_fn=train_input_fn,
    steps=NUM_STEPS)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f0bca1f3908>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into ./model/model.ckpt.
INFO:tensorflow:loss = 1178.339, step = 0
INFO

In [8]:
# Define eval_input_fn
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'x': test_x}, y=test_y, shuffle=False
)

test_acc = dnn.evaluate(input_fn=eval_input_fn, steps=1)['accuracy']
print('test accuracy: {}'.format(test_acc))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-06-27-10:03:23
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./model/model.ckpt-119
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [1/1]
INFO:tensorflow:Finished evaluation at 2018-06-27-10:03:23
INFO:tensorflow:Saving dict for global step 119: accuracy = 0.9453125, average_loss = 0.2138238, global_step = 119, loss = 27.369446
test accuracy: 0.9453125


### 7.2.3 tf.feature_column

텐서플로는 [`tf.feature_column`](https://www.tensorflow.org/api_docs/python/tf/feature_column)를 통해 다양한 데이터의 